In [1]:
import requests, os, zipfile

def download_file(id, destination):
    URL = 'https://docs.google.com/uc?export=download'

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [3]:
data_directory = 'data2'
datasets = {
    'BSDS100.zip': '1EWEsfsgElkNvOcJwZLDe2TeDIMhr6SpH',
    'BSDS200.zip': '1hIYAIODXT6GksNuk0EHiwgnVxZlDHUAI',
    'General100.zip': '1Y4R8STXyPcOOykutbAJpMsH5O5n2NdFF',
    'historical.zip': '17Rq-4gm1_rJX3KB2jolcqMGWiSSmQIWz',
    'manga109.zip': '15cAVM4BJtSGpduLufqDqfQV75m-Pfepi',
    'Set5.zip': '1RtyIeUFTyW8u7oa4z7a0lSzT3T1FwZE9',
    'Set14.zip': '1vsw07sV8wGrRQ8UARe2fO5jjgy9QJy_E',
    'T91.zip': '1dfsToAYgecVARKjw2wtQS5tsn6pzG6pr',
    'urban100.zip': '1XaY-tnBP_z21WKgOCeXBa9r-KJyBMbgZ'
}
os.mkdir(data_directory)

for dataset in datasets:
    file_id = datasets[dataset]
    file = data_directory + '/' + dataset
    print("Downloading: " + dataset)
    download_file(file_id, file)
    